# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.   // Looks like there must be forsetfires.csv

In [109]:
import pandas as pd
from logger import get_logger

_logs = get_logger(__name__)

columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('./data/fires/forestfires.csv', header = None, names = columns))
_logs.info(fires_dt.head())


2024-07-05 15:54:01,781, 3969106436.py, 10, INFO,   coord_x coord_y  month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain  \
0       X       Y  month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain   
1       7       5    mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7     0   
2       7       4    oct  tue  90.6  35.4  669.1  6.7    18  33   0.9     0   
3       7       4    oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3     0   
4       8       6    mar  fri  91.7  33.3   77.5    9   8.3  97     4   0.2   

   area  
0  area  
1     0  
2     0  
3     0  
4     0  
2024-07-05 15:54:01,781, 3969106436.py, 10, INFO,   coord_x coord_y  month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain  \
0       X       Y  month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain   
1       7       5    mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7     0   
2       7       4    oct  tue  90.6  35.4  669.1  6.7    18  33   0.9     0   
3       7       4    oct  sat  90.6  43.7  686.9 

# Get X and Y

Create the features data frame and target data.

In [110]:
from sklearn.model_selection import train_test_split

X = fires_dt.drop('area', axis = 1)
Y = fires_dt['area']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

_logs.info(X_train.head())


2024-07-05 15:54:01,988, 2822058215.py, 8, INFO,     coord_x coord_y month  day  ffmc    dmc     dc  isi  temp  rh wind rain
405       2       4   sep  wed  87.9   84.8  725.1  3.7  21.8  34  2.2    0
331       3       3   sep  sat  92.2  102.3  751.5  8.4  24.2  27  3.1    0
220       6       5   mar  mon  90.1   39.7   86.6  6.2  15.2  27  3.1    0
148       8       3   sep  tue  84.4   73.4  671.9  3.2  24.2  28  3.6    0
301       6       5   jun  mon  90.4   93.3  298.1  7.5  20.7  25  4.9    0
2024-07-05 15:54:01,988, 2822058215.py, 8, INFO,     coord_x coord_y month  day  ffmc    dmc     dc  isi  temp  rh wind rain
405       2       4   sep  wed  87.9   84.8  725.1  3.7  21.8  34  2.2    0
331       3       3   sep  sat  92.2  102.3  751.5  8.4  24.2  27  3.1    0
220       6       5   mar  mon  90.1   39.7   86.6  6.2  15.2  27  3.1    0
148       8       3   sep  tue  84.4   73.4  671.9  3.2  24.2  28  3.6    0
301       6       5   jun  mon  90.4   93.3  298.1  7.5  20.7  25 

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [111]:
# preproc1
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

preproc1 = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

print(X_train.head())

    coord_x coord_y month  day  ffmc    dmc     dc  isi  temp  rh wind rain
405       2       4   sep  wed  87.9   84.8  725.1  3.7  21.8  34  2.2    0
331       3       3   sep  sat  92.2  102.3  751.5  8.4  24.2  27  3.1    0
220       6       5   mar  mon  90.1   39.7   86.6  6.2  15.2  27  3.1    0
148       8       3   sep  tue  84.4   73.4  671.9  3.2  24.2  28  3.6    0
301       6       5   jun  mon  90.4   93.3  298.1  7.5  20.7  25  4.9    0


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [129]:
# preproc2
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
import numpy as np

# area can't be transformed as this column is in our Y dataset
#numeric_features_no_transform = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']

#numeric_features_to_transform = ['area']


def log_transform(x):
    return np.log1p(x)

log_transformer = Pipeline(steps=[
    #('log', FunctionTransformer(log_transform, validate=True)),
    ('num', Pipeline([
        ('scaler', StandardScaler()),
        ('transformer', PowerTransformer(method='yeo-johnson'))
    ]), numeric_features),
    ('scale', StandardScaler())
])

preproc2 = ColumnTransformer(
    #transformers=[
    #    #('num', StandardScaler(), numeric_features_no_transform),
    #    ('num_transform', log_transformer, numeric_features),
    #    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    #]
    transformers=[
        ('num', Pipeline([
            ('scaler', StandardScaler()),
            ('transformer', PowerTransformer(method='yeo-johnson'))
        ]), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [130]:
# Pipeline A = preproc1 + baseline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

baseline = LinearRegression()
advanced = RandomForestRegressor()

pipeline_a = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', baseline)
])

In [131]:
# Pipeline B = preproc2 + baseline
pipeline_b = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', baseline)
])

In [133]:
# Pipeline C = preproc1 + advanced model

pipeline_c = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', advanced)
])

In [132]:
# Pipeline D = preproc2 + advanced model
pipeline_d = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', advanced)
])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [143]:
scoring = {
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'r2': 'r2'
}

In [144]:
from sklearn.model_selection import GridSearchCV

params_lr = {
    'regressor__fit_intercept': [True, False],
    'regressor__copy_X': [True, False],
    'regressor__n_jobs': [None, -1],
    'regressor__positive': [True, False]
}

params_rf = {
    'regressor__bootstrap': [True, False],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__n_estimators': [10, 50, 100, 200],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
}

def grid_search_pipeline(pipeline, param_grid, X_train, y_train, scoring):
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=scoring, refit='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search

grid_search_a = grid_search_pipeline(pipeline_a, params_lr, X_train, Y_train, scoring)
grid_search_b = grid_search_pipeline(pipeline_b, params_lr, X_train, Y_train, scoring)
grid_search_c = grid_search_pipeline(pipeline_c, params_rf, X_train, Y_train, scoring)
grid_search_d = grid_search_pipeline(pipeline_d, params_rf, X_train, Y_train, scoring)


In [149]:
print("Pipeline A best params: ", grid_search_a.best_params_)
print("Pipeline A best score: ", grid_search_a.best_score_)
print("Pipeline B best params: ", grid_search_b.best_params_)
print("Pipeline B best score: ", grid_search_b.best_score_)
print("Pipeline C best params: ", grid_search_c.best_params_)
print("Pipeline C best score: ", grid_search_c.best_score_)
print("Pipeline D best params: ", grid_search_d.best_params_)
print("Pipeline D best score: ", grid_search_d.best_score_)

Pipeline A best params:  {'regressor__copy_X': True, 'regressor__fit_intercept': False, 'regressor__n_jobs': None, 'regressor__positive': True}
Pipeline A best score:  -5229.156343070289
Pipeline B best params:  {'regressor__copy_X': True, 'regressor__fit_intercept': False, 'regressor__n_jobs': None, 'regressor__positive': True}
Pipeline B best score:  -5245.555526689857
Pipeline C best params:  {'regressor__bootstrap': True, 'regressor__max_depth': 20, 'regressor__min_samples_leaf': 4, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 10}
Pipeline C best score:  -5317.033862078361
Pipeline D best params:  {'regressor__bootstrap': True, 'regressor__max_depth': 30, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 50}
Pipeline D best score:  -5367.0046425793325


# Evaluate

+ Which model has the best performance?

## Results

Pipeline A (Linear Regression with StandardScaler):

Best Params: {'regressor__copy_X': True, 'regressor__fit_intercept': False, 'regressor__n_jobs': None, 'regressor__positive': True}
Best Score (Negative MSE): -5229.156343070289
Pipeline B (Linear Regression with MinMaxScaler):

Best Params: {'regressor__copy_X': True, 'regressor__fit_intercept': False, 'regressor__n_jobs': None, 'regressor__positive': True}
Best Score (Negative MSE): -5245.555526689857
Pipeline C (Random Forest with StandardScaler):

Best Params: {'regressor__bootstrap': True, 'regressor__max_depth': 20, 'regressor__min_samples_leaf': 4, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 10}
Best Score (Negative MSE): -5317.033862078361
Pipeline D (Random Forest with MinMaxScaler):

Best Params: {'regressor__bootstrap': True, 'regressor__max_depth': 30, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 50}
Best Score (Negative MSE): -5367.0046425793325

From these results, Pipeline A seems to have the best performance, with the highest (least negative) score.




# Export

+ Save the best performing model to a pickle file.

In [150]:
import pickle

best_params = grid_search_a.best_params_
print(f'Best Hyperparameters: {best_params}')


with open('best_model.pkl', 'wb') as model_file:
    pickle.dump(grid_search_a, model_file)


Best Hyperparameters: {'regressor__copy_X': True, 'regressor__fit_intercept': False, 'regressor__n_jobs': None, 'regressor__positive': True}


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
import shap

# I'll finish soon. Couldn't finish on Sunday

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.